In [1]:
import pandas as pd
import numpy as np
import py_stringmatching as sm
import py_stringsimjoin as ssj
from pandas_profiling import ProfileReport
import re
import nltk
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
bx = pd.read_csv('../../datasets/books/bx.csv',dtype='str',sep=';')

In [3]:
half = pd.read_csv('../../datasets/books/half.csv',dtype='str',sep=';')

In [4]:
bx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270948 entries, 0 to 270947
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   bx_id                          270948 non-null  object
 1   bx_isbn                        270948 non-null  object
 2   bx_title                       270948 non-null  object
 3   bx_firstauthor                 270947 non-null  object
 4   bx_pubdate                     270948 non-null  object
 5   bx_pages                       270948 non-null  object
 6   bx_publisher                   270946 non-null  object
 7   bx_binding                     4171 non-null    object
 8   bx_title_no_par                270946 non-null  object
 9   bx_title_no_par_no_stop_words  270658 non-null  object
dtypes: object(10)
memory usage: 20.7+ MB


In [5]:
half.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3099 entries, 0 to 3098
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   half_id                          3099 non-null   object
 1   half_isbn                        3099 non-null   object
 2   half_title                       3099 non-null   object
 3   half_firstauthor                 3094 non-null   object
 4   half_binding                     460 non-null    object
 5   half_pubdate                     3028 non-null   object
 6   half_pages                       3099 non-null   object
 7   half_publisher                   3099 non-null   object
 8   half_title_no_par                3099 non-null   object
 9   half_title_no_par_no_stop_words  3094 non-null   object
dtypes: object(10)
memory usage: 242.2+ KB


In [6]:
bx.fillna('',inplace=True)
half.fillna('',inplace=True)

In [7]:
matches_bx_half = pd.merge(bx,half,left_on='bx_isbn',right_on='half_isbn')
print('True matches across bx and half: {}'.format(matches_bx_half.shape[0]))

True matches across bx and half: 550


In [8]:
half.columns

Index(['half_id', 'half_isbn', 'half_title', 'half_firstauthor', 'half_binding', 'half_pubdate', 'half_pages', 'half_publisher', 'half_title_no_par', 'half_title_no_par_no_stop_words'], dtype='object')

In [9]:
bx.columns

Index(['bx_id', 'bx_isbn', 'bx_title', 'bx_firstauthor', 'bx_pubdate', 'bx_pages', 'bx_publisher', 'bx_binding', 'bx_title_no_par', 'bx_title_no_par_no_stop_words'], dtype='object')

### Join on author based on Jaccard score of at least 0.3 with 3-gram tokenization

In [10]:
q3 = sm.QgramTokenizer(qval=3,return_set=True)

In [14]:
bx_half_pairs_a_q3_jac_03 = ssj.jaccard_join(bx, half, 'bx_id', 'half_id', 'bx_author_lower', 'half_author_lower', q3, 0.3, 
                                     l_out_attrs=['bx_title','bx_title_lower','bx_title_no_par','bx_title_no_par_no_stop_words','bx_author','bx_author_lower','bx_binding','bx_pubdate', 'bx_pages','bx_publisher','bx_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Join on title (with parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [15]:
bx_half_pairs_tit_q3_jac_03 = ssj.jaccard_join(bx, half, 'bx_id', 'half_id', 'bx_title_lower', 'half_title_lower', q3, 0.3, 
                                     l_out_attrs=['bx_title','bx_title_lower','bx_title_no_par','bx_title_no_par_no_stop_words','bx_author','bx_author_lower','bx_binding','bx_pubdate', 'bx_pages','bx_publisher','bx_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Join on title (without parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [16]:
bx_half_pairs_titnp_q3_jac_03 = ssj.jaccard_join(bx, half, 'bx_id', 'half_id', 'bx_title_no_par', 'half_title_no_par', q3, 0.3, 
                                     l_out_attrs=['bx_title','bx_title_lower','bx_title_no_par','bx_title_no_par_no_stop_words','bx_author','bx_author_lower','bx_binding','bx_pubdate', 'bx_pages','bx_publisher','bx_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

/home/debian/anaconda3/envs/mag/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


### Join on title (without parentheses and stop words) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [17]:
bx_half_pairs_titnp_nosw_q3_jac_03 = ssj.jaccard_join(bx, half, 'bx_id', 'half_id', 'bx_title_no_par_no_stop_words', 'half_title_no_par_no_stop_words', q3, 0.3, 
                                     l_out_attrs=['bx_title','bx_title_lower','bx_title_no_par','bx_title_no_par_no_stop_words','bx_author','bx_author_lower','bx_binding','bx_pubdate', 'bx_pages','bx_publisher','bx_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Evaluate Blocking

In [18]:
print('True matches across bx and half: {}'.format(matches_bx_half.shape[0]))

True matches across bx and half: 550


In [19]:
def assignTrueLabels(df_blockedpairs, idcolumn1, idcolumn2):
    return df_blockedpairs.apply(lambda row: 1 if (row[idcolumn1]==row[idcolumn2]) else 0, axis=1)

In [20]:
def calcRedRatio(df1,df2,df_blockedpairs):
    cp = (df1.shape[0]*df2.shape[0])/2
    rr = (1-(df_blockedpairs.shape[0]/cp))*100
    return round(rr,2)

In [21]:
def calcPairCompISBN(df_matches,df_blockedpairs,true_label_col):
    pc = (df_blockedpairs[df_blockedpairs[true_label_col]==1].shape[0]/df_matches.shape[0])*100
    return round(pc,2)

In [22]:
#Assign labels to the blockedpairs df
bx_half_pairs_a_q3_jac_03['true_label'] = assignTrueLabels(bx_half_pairs_a_q3_jac_03,'l_bx_isbn','r_half_isbn')
bx_half_pairs_tit_q3_jac_03['true_label'] = assignTrueLabels(bx_half_pairs_tit_q3_jac_03,'l_bx_isbn','r_half_isbn')
bx_half_pairs_titnp_q3_jac_03['true_label'] = assignTrueLabels(bx_half_pairs_titnp_q3_jac_03,'l_bx_isbn','r_half_isbn')
bx_half_pairs_titnp_nosw_q3_jac_03['true_label'] = assignTrueLabels(bx_half_pairs_titnp_nosw_q3_jac_03,'l_bx_isbn','r_half_isbn')

In [23]:
print('Blocking based on author, Jaccaed, 3-gram, Threshold 0.3')
print('Evaluation of bx_half_pairs_a_q3_jac_03')
print('Number of rows: {:,}'.format(bx_half_pairs_a_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(bx,half,bx_half_pairs_a_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_bx_half,bx_half_pairs_a_q3_jac_03,'true_label')))

print('\n')
print('Evaluation of bx_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(bx_half_pairs_tit_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(bx,half,bx_half_pairs_tit_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_bx_half,bx_half_pairs_tit_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of bx_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(bx_half_pairs_titnp_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(bx,half,bx_half_pairs_titnp_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_bx_half,bx_half_pairs_titnp_q3_jac_03,'true_label')))

print('\n')
print('Evaluation of bx_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(bx_half_pairs_titnp_nosw_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(bx,half,bx_half_pairs_titnp_nosw_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_bx_half,bx_half_pairs_titnp_nosw_q3_jac_03,'true_label')))

Blocking based on author, Jaccaed, 3-gram, Threshold 0.3
Evaluation of bx_half_pairs_a_q3_jac_03
Number of rows: 700,881
Reduction Ratio: 99.83%
Pair Completeness: 99.27%


Evaluation of bx_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)
Number of rows: 72,086
Reduction Ratio: 99.98%
Pair Completeness: 89.82%


Evaluation of bx_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 123,636
Reduction Ratio: 99.97%
Pair Completeness: 88.73%


Evaluation of bx_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 141,942
Reduction Ratio: 99.97%
Pair Completeness: 89.45%


In [24]:
bx_half_pairs_a_q3_jac_03.to_csv('filtered_correspondences/bx_half_pairs_a_q3_jac_03.csv',index=False)

In [20]:
matches_bx_half.to_csv('matching_pairs/matches_bx_half.csv',index=False)

-- Blocking based on author with Jaccard distance using 3-gram tokens performs best.